In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# 한글 안 쓰므로 폰트 설정은 생략
plt.rcParams['axes.unicode_minus'] = False
sns.set(style="whitegrid")

# 0. 엑셀 파일 불러오기
# - 파일명은 merged_data.xlsx
# - 현재 실행 중인 파이썬 파일과 같은 폴더에 있어야 함
df = pd.read_excel("merged_data.xlsx")

# 1. 통합지표 계산 (total_risk)
# - 사망자수, 중상자수, 경상자수는 반드시 존재해야 하며 결측치 없어야 함
# - 실수/정수형 타입으로 변환 후 계산
if 'total_risk' not in df.columns:
    df['사망자수'] = pd.to_numeric(df['사망자수'], errors='coerce').fillna(0)
    df['중상자수'] = pd.to_numeric(df['중상자수'], errors='coerce').fillna(0)
    df['경상자수'] = pd.to_numeric(df['경상자수'], errors='coerce').fillna(0)

    df['incident_score'] = 1.0 + 0.7 * (df['중상자수'] > 0) + 0.3 * (df['경상자수'] > 0)
    df['severity_score'] = df['사망자수'] * 1.0 + df['중상자수'] * 0.7 + df['경상자수'] * 0.3
    df['total_risk'] = df['incident_score'] * 0.4 + df['severity_score'] * 0.6

# 2. 수치형 변수 목록 정의
num_cols = ['사망자수', '중상자수', '경상자수', '부상신고자수',
            'incident_score', 'severity_score', 'total_risk']

# 3. 히트맵
# - 변수들 간의 상관관계를 한눈에 확인하기 위한 시각화
# - 각 변수의 연관 정도를 파악해 모델 입력 후보 선정 가능
plt.figure(figsize=(10, 8))
sns.heatmap(df[num_cols].corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Correlation Heatmap")
plt.tight_layout()
plt.show()

# 4. pairplot
# - 변수 간 산점도 + 분포를 함께 볼 수 있음
# - 변수 간 관계 패턴, 이상치 유무, 군집 가능성 탐색용
# - 5천 건 이상이면 샘플링 필요
df_sample = df.sample(5000, random_state=42) if len(df) > 5000 else df
sns.pairplot(df_sample[num_cols])
plt.suptitle("Pairplot", y=1.02)
plt.show()

# 5. boxplot (도로등급별 위험도 차이 확인)
# - 도로등급별로 total_risk의 분포 차이를 시각적으로 비교
# - 범주형 변수(도로등급)가 있어야 실행 가능
if '도로등급' in df.columns:
    plt.figure(figsize=(12, 6))
    sns.boxplot(data=df, x='도로등급', y='total_risk')
    plt.title("Risk by Road Grade")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

# 6. scatterplot (통행량 vs 사고건수)
# - 교통량이 많을수록 사고가 많은지 확인하는 기초 관계 분석
# - 통행량, 사고건수 컬럼은 수치형이어야 하고 결측치 없어야 함
if {'통행량', '사고건수'}.issubset(df.columns):
    df['통행량'] = pd.to_numeric(df['통행량'], errors='coerce')
    df['사고건수'] = pd.to_numeric(df['사고건수'], errors='coerce')
    plt.figure(figsize=(8, 6))
    sns.scatterplot(data=df, x='통행량', y='사고건수', hue='도로등급' if '도로등급' in df.columns else None, alpha=0.6)
    plt.title("Traffic Volume vs Accident Count")
    plt.tight_layout()
    plt.show()

# 7. barplot (Top10 위험도로)
# - 위험도가 높은 도로를 선별해 시각적으로 확인
# - 도로명 컬럼이 존재해야 함
if '도로명' in df.columns:
    top10 = df.sort_values('total_risk', ascending=False).head(10)
    plt.figure(figsize=(12, 6))
    sns.barplot(data=top10, x='도로명', y='total_risk')
    plt.title("Top 10 High Risk Roads")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

# 8. kdeplot (변수 분포 형태 파악)
# - 각 수치형 변수의 분포 확인 (왜도, 이상치 등 포함)
# - 숫자형이고 결측치 없는 값만 대상으로 실행
for col in num_cols:
    try:
        clean_series = pd.to_numeric(df[col], errors='coerce').dropna()
        if len(clean_series) > 0:
            plt.figure(figsize=(8, 4))
            sns.kdeplot(clean_series, shade=True)
            plt.title(f"{col} Distribution")
            plt.tight_layout()
            plt.show()
    except Exception as e:
        print(f"{col} skipped due to error: {e}")
